# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook has been adapted from the [**official YOLOv8 guide** by **Ultralytics**](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb) to train and evaluate YOLOv8 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the official repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

NOTE: Ultralytics version [8.0.19](https://pypi.org/project/ultralytics/8.0.19/) was used.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of five parts:
1. Setup
2. Training
3. Evaluation
4. Predict
5. Export

<img src="https://github.com/geezacoleman/Palmer-detection/assets/51358498/6040227e-a072-43bf-a789-72e0833f3168"/>

Browse the <a href="https://docs.ultralytics.com/">official YOLOv8 Docs</a> for specific issues with YOLOv8 or consider leaving an issue in this repository.


# 1. Setup

Install `ultralytics` from our cloned repository (ensuring to replicate version [8.0.19](https://pypi.org/project/ultralytics/8.0.19/))and [dependencies](https://github.com/ultralytics/ultralytics/blob/master/requirements.txt) and check PyTorch and GPU use.

More recent versions of Ultralytics are available via `pip install ultralytics`.

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import necessary packages
import torch
import random

random.seed(0)

from datetime import datetime

DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%cd '/content/drive/MyDrive/Palmer-detection/ultralytics'

In [ ]:
# Git clone method (used for development and replication of results)
%pip install -qe ultralytics
import ultralytics
ultralytics.checks()

Note this will be version [8.0.19](https://pypi.org/project/ultralytics/8.0.19/) to replicate results in the study. For the latest version, please check official Ultralytics documentation.

# 2. Train

Train YOLOv8 on detection, segmentation and classification datasets.

In [ ]:
!yolo task=init

In [ ]:
models = ['n', 'm', 'x', 'x6']
classes = ['1', '8']
%cd '/content/drive/MyDrive/Palmer-detection/ultralytics'

for model in models:
  for class_num in classes:
    for fold in range(0, 5):
      yaml_path = f"{class_num}cls_fold_{fold}.yaml"
      full_name = f'train/{DATE}_yolov8{model}_B8_F{fold}_{class_num}cls'

      !yolo task=detect mode=train model=yolov8{model}.pt data={yaml_path} epochs=30 imgsz={IMAGE_SIZE} batch=8 name={full_name} augment=True

# 3. Evaluation
YOLOv8 does not automatically use the test split when evaluating model performance and there is no option to specify this in their package. Thus the following functions must be used to rename the val split within the .yaml files so that the test split is used.

In [ ]:
def replace_and_save(file_path):
    with open(file_path, 'r') as f:
        text = f.read()
    text = text.replace('val', 'IGNORE')
    text = text.replace('test:', 'val:')
    new_file_path = file_path.replace('.yaml', '_test.yaml')
    with open(new_file_path, 'w') as f:
        f.write(text)

In [ ]:
import os
%cd '/content/drive/My Drive/Palmer-detection/ultralytics'
yamls = r'ultralytics/yolo/data/datasets/testing'
for yaml in os.listdir(yamls):
  print(yaml)
  replace_and_save(os.path.join(yamls, yaml))

Run the evaluation script for everything within the train directory.

In [ ]:
import os
train_path = "/content/drive/My Drive/Palmer-detection/ultralytics/runs/detect/train"
%cd '/content/drive/My Drive/Palmer-detection/ultralytics'

for run_name in os.listdir(train_path):
  if 'v8' not in run_name:
    print(f'skipping {run_name}')
  
  else:
    fold = run_name.split("_")[-3][-1]
    group = run_name.split("_")[-2][:-3]
    
    yaml_path = f"{class_num}cls_fold_{fold}.yaml"
    out_name = 'val/' + run_name + '_val' # save name into the val directory as this is not automatic
  
    !yolo task=detect mode=val model=runs/detect/train/{run_name}/weights/best.pt data={yaml_path} imgsz={IMAGE_SIZE} name={out_name} batch=8 conf=0.001 iou=0.6
  

# 4. Predict

`yolo mode=predict` runs YOLOv8 inference on a variety of sources, downloading models automatically from the [latest YOLOv8 release](https://github.com/ultralytics/ultralytics/releases), and saving results to `runs/predict`.


In [ ]:
full_name = 'ENTER RUN NAME HERE'
!yolo task=detect mode=predict model=runs/detect/{full_name}/weights/best.pt conf=0.25 source=datasets/test imgsz=1280
# display.Image(filename='runs/detect/predict/zidane.jpg', width=600)

# 5. Export

Export a YOLOv8 model to any format.

Format                  | `format=...**bold text**`         | Model
---                     | ---                       | ---
PyTorch                 | -                         | yolov8n.pt
TorchScript             | `torchscript`             | yolov8n.torchscript
ONNX                    | `onnx`                    | yolov8n.onnx
OpenVINO                | `openvino`                | yolov8n_openvino_model/
TensorRT                | `engine`                  | yolov8n.engine
CoreML                  | `coreml`                  | yolov8n.mlmodel
TensorFlow SavedModel   | `saved_model`             | yolov8n_saved_model/
TensorFlow GraphDef     | `pb`                      | yolov8n.pb
TensorFlow Lite         | `tflite`                  | yolov8n.tflite
TensorFlow Edge TPU     | `edgetpu`                 | yolov8n_edgetpu.tflite
TensorFlow.js           | `tfjs`                    | yolov8n_web_model/
PaddlePaddle            | `paddle`                  | yolov8n_paddle_model/

In [ ]:
!yolo mode=export model=yolov8n.pt format=torchscript